In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, AutoTokenizer
import pandas as pd
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

In [2]:
separator = " = /@\ = "

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

In [6]:
dataset = load_dataset('csv', data_files={'train': '../data/processed/train.csv',
                                          'test': '../data/processed/test.csv',
                                          'validation': '../data/processed/val.csv'})
dataset

Using custom data configuration default-067da67f4370b087


100%|██████████| 3/3 [00:00<00:00, 743.63it/s]


Dataset csv downloaded and prepared to /home/pheithar/.cache/huggingface/datasets/csv/default-067da67f4370b087/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 468.43it/s]


DatasetDict({
    train: Dataset({
        features: ['entry_name', 'name', 'description'],
        num_rows: 8409
    })
    test: Dataset({
        features: ['entry_name', 'name', 'description'],
        num_rows: 2803
    })
    validation: Dataset({
        features: ['entry_name', 'name', 'description'],
        num_rows: 2803
    })
})

In [ ]:
max(len(x.split(" ")) for x in dataset["train"]["description"])

In [ ]:
def tokenize_function(examples):
    output = [separator + pkmn_name for pkmn_name in examples["name"]]

    results = tokenizer(examples["description"], output, max_length=65, padding="max_length")
    results["labels"] = results["input_ids"].copy()
    return results

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenizer.decode(tokenized_datasets["train"]["input_ids"][0])

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10)).remove_columns(["name", "description", "entry_name"])
# small_train_dataset = small_train_dataset.rename_column("name", "label")

In [ ]:
small_train_dataset

In [ ]:
training_args = TrainingArguments("test_trainer", label_names=None)

In [ ]:
def compute_metrics(eval_pred):
    print(eval_pred)


trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset
)

In [ ]:
trainer.train()

In [ ]:
"Its flames are strong enough to melt iron bars."+separator

In [ ]:
sample = tokenizer.encode("Its can breathe under water."+separator, max_length=60, return_tensors="pt")
sample

In [ ]:
result = model.generate(sample)
result

In [ ]:
tokenizer.decode(result[0], skip_special_tokens=True)

In [ ]:
model.save_pretrained("../models/")